In [ ]:
from pydna.dseqrecord import Dseqrecord
# Let's create a DNA sequence record, and add a feature to it
dsr = Dseqrecord("ATGCAAACAGTAATGATGGATGACATTCAAAGCACTGATTCTATTGCTGAAAAAGATAAT")
dsr.add_feature(x=0, y=60,type="gene", label="my_gene") # We add a feature to highlight the sequence as a gene
dsr.figure()


Dseqrecord(-60)
ATGCAAACAGTAATGATGGATGACATTCAAAGCACTGATTCTATTGCTGAAAAAGATAAT
TACGTTTGTCATTACTACCTACTGTAAGTTTCGTGACTAAGATAACGACTTTTTCTATTA

In [ ]:
# This is how it would look as a genbank file
print(dsr.format("genbank"))

LOCUS       name                      60 bp    DNA     linear   UNK 01-JAN-1980
DEFINITION  description.
ACCESSION   id
VERSION     id
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     misc            1..60
                     /type="gene"
                     /label="my_gene"
ORIGIN
        1 atgcaaacag taatgatgga tgacattcaa agcactgatt ctattgctga aaaagataat
//


In [ ]:
# Now let's design primers to amplify it
from pydna.design import primer_design
# limit is the minimum length of the primer, target_tm is the desired melting temperature of the primer
amplicon = primer_design(dsr, limit=13, target_tm=55)
# Let's print the primers, and a figure that shows where they align with the template sequence
print("forward primer:", amplicon.forward_primer.seq)
print("reverse primer:", amplicon.reverse_primer.seq)
amplicon.figure()

forward primer: ATGCAAACAGTAATGATGGA
reverse primer: ATTATCTTTTTCAGCAATAGAATCA


5ATGCAAACAGTAATGATGGA...TGATTCTATTGCTGAAAAAGATAAT3
                        |||||||||||||||||||||||||
                       3ACTAAGATAACGACTTTTTCTATTA5
5ATGCAAACAGTAATGATGGA3
 ||||||||||||||||||||
3TACGTTTGTCATTACTACCT...ACTAAGATAACGACTTTTTCTATTA5

In [ ]:
# Let's say we don't want to just amplify it, but we want to add restriction sites to it!

from pydna.amplify import pcr
# We add the restriction sites to the primers
forward_primer = "ccccGGATCC" + amplicon.forward_primer
reverse_primer = "ttttGGATCC" + amplicon.reverse_primer

# We do the PCR
pcr_product = pcr(forward_primer, reverse_primer, dsr)
# The PCR product is of class `Amplicon`, a subclass of `Dseqrecord`.
# When doing a figure, it shows where primers anneal.
pcr_product.figure()

          5ATGCAAACAGTAATGATGGA...TGATTCTATTGCTGAAAAAGATAAT3
                                  |||||||||||||||||||||||||
                                 3ACTAAGATAACGACTTTTTCTATTACCTAGGtttt5
5ccccGGATCCATGCAAACAGTAATGATGGA3
           ||||||||||||||||||||
          3TACGTTTGTCATTACTACCT...ACTAAGATAACGACTTTTTCTATTA5

In [ ]:
# If we want to see the sequence more clearly, we can turn it into a `Dseqrecord`
pcr_product = Dseqrecord(pcr_product)

print(pcr_product.format("genbank"))

pcr_product.figure()

LOCUS       m_y___g_e_n_e             80 bp    DNA     linear   UNK 01-JAN-1980
DEFINITION  pcr_product_f60 id_r60 id.
ACCESSION   m_y___g_e_n_e
VERSION     m_y___g_e_n_e
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     misc            11..70
                     /type="gene"
                     /label="my_gene"
     primer_bind     11..30
                     /label="f60"
                     /PCR_conditions="primer
                     sequence:ccccGGATCCATGCAAACAGTAATGATGGA"
                     /ApEinfo_fwdcolor="#baffa3"
                     /ApEinfo_revcolor="#ffbaba"
     primer_bind     complement(46..70)
                     /label="r60"
                     /PCR_conditions="primer
                     sequence:ttttGGATCCATTATCTTTTTCAGCAATAGAATCA"
                     /ApEinfo_fwdcolor="#baffa3"
                     /ApEinfo_revcolor="#ffbaba"
ORIGIN
        1 ccccggatcc atgcaaacag taatgatgga tgacattcaa agcactgatt ctattgctg

Dseqrecord(-80)
ccccGGATCCATGCAAACAGTAATGATGGATGACATTCAAAGCACTGATTCTATTGCTGAAAAAGATAATGGATCCaaaa
ggggCCTAGGTACGTTTGTCATTACTACCTACTGTAAGTTTCGTGACTAAGATAACGACTTTTTCTATTACCTAGGtttt

In [ ]:

5atg...taa3 |||
5ccccGGATCCatg3
3attCCTAGGtttt5
|||
3tac...att5
In [5]: Dseqrecord (pcr_product).figure()
Out [5]: Dseqrecord(-29)
ccccGGATCCatgccctaaGGATCCaaaa ggggCCTAGGtacgggattCCTAGGtttt
In [6]: from Bio. Restriction import BamHI # cuts GGATCC a, b, c = pcr product.cut (BamHI)
print(a.figure())
print()
print (b.figure())
print()
print(c.figure())
Dseqrecord(-9)
CCCCG
ggggCCTAG
Dseqrecord(-19)
GATCCatgccctaaG GtacgggattCCTAG
Dseqrecord(-9)
GATCCaaaa
Gtttt
In [7]: vector = Dseqrecord("aatgtttttccctCCCGGGcaaaatAGATCTtgctatgcatcatcgatct"
circular=True, name="vect")
In [8]: vector.figure()
Out[8]: Dseqrecord (050)
aatgtttttccctCCCGGGcaaaatAGATCTtgctatgcatcatcgatct ttacaaaaagggaGGGCCCgttttaTCTAGAacgatacgtagtagctaga
In [9]: from Bio. Restriction import BglII # cuts AGATCT
linear_vector_bgl vector.linearize (BglII)
rec_vector= (linear_vector_bgl + b).looped().synced(vector)
rec_vector.figure()
Out [9]: Dseqrecord (065)
aatgtttttccctCCCGGGcaaaatAGATCCatgccctaaGGATCTtgctatgcatcatcgatct ttacaaaaagggaGGGCCCgttttaTCTAGGtacgggattCCTAGAacgatacgtagtagctaga
In [10]: gene2 = Dseqrecord("cctCCCatgaaataaGGGcaa", name="gene2") gene2.add_feature (6,15)
gene2.figure()
Out[10]: Dseqrecord(-21)
cctCCCatgaaataaGGGcaa
ggaGGGtactttattCCCgtt
In [11]: from pydna.assembly import Assembly
from Bio. Restriction import SmaI # cuts CCCGGG
linear_vector_sma = vector.linearize(SmaI)
asm = Assembly ((linear_vector_sma, gene2), limit=6)
candidate, *rest = asm.assemble_circular()
In [12]: candidate.figure()
Out[12] - vect_lin❘ 6
\/
^\
6 gene2 6
V
ハ
6-
In [13]: candidate candidate.synced (vector, limit=10)
In [14]: Dseqrecord (candidate).figure()
Out[14]: Dseqrecord(059)
aatgtttttccctCCCatgaaataaGGGcaaaatAGATCTtgctatgcatcatcgatct ttacaaaaagggaGGGtactttattCCcgttttaTCTAGAacgatacgtagtagctaga
In [16]: from pydna.gel import gel
from pydna.ladders import GeneRuler_1kb_plus
In [17] band Dseqrecord("GATC"*500)
In [18]: gel([ GeneRuler_1kb_plus,
Out [18]: 20000 -
[band, ]])
10000 -
7000
5000
4000
-
3000
-
2000
1500
1000
700
500
400
300
200
75

SyntaxError: invalid decimal literal (861700443.py, line 1)